# 들어가며

이제 해야할 일은 각각의 집계구 폴리곤에 대해 그 내부에 연령대별 인구만큼 점을 만드는 것입니다. 그리고 그 점마다 어느 연령대에 속하는 점인지 알 수 있게끔 해줘야할 것입니다. 

우리가 이전 단계에서 만든 tract_all.geojson에는 연령대별 인구를 보여주는 column이 있습니다. in_age_001, in_age_002,...,in_age_021이 각각 5년 단위의 연령대별 인구를 나타내는 column입니다. ArcMap, QGIS, PostGIS에는 dataset의 한 column의 숫자만큼 해당 row의 polygon 내부의 임의의 위치에 point를 생성하는 명령이 있습니다. 그런데 우리는 여러 컬럼에 대해 이 작업을 해야하고 한번 작업을 할 때마다 새로 생성된 점들에 labeling을 해줘야합니다. 그렇지 않다면 나중에 어느 점이 어느 연령대의 점인지 알 수 없을테니까요. 그리고 우리는 최대한 command line에서 문제를 처리하고 싶습니다.

## dot-density 
그래서 찾은 것이 [dot-density](https://github.com/anandthakker/dot-density)입니다. 이 툴은 node.js로 작성된 것인데 geojson을 받아들여서 한 컬럼에 지정된 숫자만큼 polygon 내부에 점을 생성시킨뒤 결과물을 geojson으로 내보냅니다. 

> cat someCensusData.geojson | dot-density --population POP10 --sampleRate 0.1

이렇게 하면 한 센서스 데이터 geojson에서 POP10 컬럼에 있는 숫자에 대해 0.1의 비율로 점을 샘플링하는 것입니다. 우리의 데이터로 바꿔 쓰면,

> cat tract_all.geojson | dot-density --population in_age_001 --sampleRate 1.0 > dot_001.geojson

이렇게 하면 우리가 앞서 만든 tract_all.geojson을 읽어서 in_age_001 (0-4세)의 숫자만큼 점을 생성하고 결과물이 dot_001.geojson에 저장되는 것입니다. 이 명령을 001부터 021까지 반복한 다음에 결과물 geojson들을 합치면 되겠지요. 그런데! 그냥 합치면 안되고 각각의 점들이 어느 연령대를 나타내는지 label이 붙어 있어야할 것입니다. 그래서 저는 [dot-density를 살짝 수정](https://github.com/yonghah/dot-density)해서 command line argument로 value를 줄 수 있게 했습니다. 그리고 그 value가 geojson의 property가 되게 했습니다. 

> cat tract_all.geojson | dot-density --population in_age_004 --sampleRate 1.0 --value 4 > dot_004.geojson

이렇게 하면 dot_004.geojson의 모든 점들은 4라는 값을 'value'라는 property에 대해 가지게 되어서 나중에 geojson을 합치더라도 어느 연령대의 점인지 구분할 수 있게 됩니다. 아래의 간단한 bash script를 이용하면 손쉽게 21개 컬럼에 대해 labeling된 point들을 얻을 수 있습니다. 1시간 정도 걸리는 작업이고 최종적으로 생성된 21개의 geojson은 6GB 정도 됩니다.

In [ ]:
%%bash
polygon=$( cat ../processed-data/tract_all.geojson )
for ((i=1; i<=21; i++)); do
	num=$(printf '%03d' $i)
	field='in_age_'$num
	echo $polygon | dot-density --population $field --sampleRate 1 --value $i > ../processed-data/partial/dot_$num.geojson
done

# 모두 합쳐서 dataframe 만들기

이제 위의 과정에서 생성된 5천만개의 포인트들이 21개에 나눠 담겨져 있는 geojson들을 하나로 합치는 작업을 하겠습니다. 합치면서 좌표도 easting하고 northing으로 바꿀겁니다. 

In [2]:
# 일단 library를 import합시다.
import pandas as pd
import geopandas as gpd
from datashader.utils import lnglat_to_meters as webm

아래의 함수는 geojson 파일을 Geopandas의 geodataframe으로 읽은 뒤, 좌표를 easting과 northing으로 변환하여 value (연령대)와 함께 반환합니다. 이 함수를 앞서의 21개 geojson에 대해 실행하고 합치면 되겠죠.

In [3]:
def get_ds_df(gj_path):
    df = gpd.read_file(gj_path)
    df = df.assign(easting = lambda x: x.geometry.map(lambda p: webm(p.x, p.y)[0]))
    df = df.assign(northing = lambda x: x.geometry.map(lambda p: webm(p.x, p.y)[1]))
    df.value = df.value.astype('category')
    return pd.DataFrame(df[['easting', 'northing', 'value']])

이제 합칩니다. 시간이 조금 걸립니다.

In [4]:
%%time
geojsons = list()
for i in range(1,22):
    gj = "../processed-data/partial/" + 'dot_{0:03d}.geojson'.format(i)
    geojsons.append(get_ds_df(gj))
print(geojsons)

[              easting      northing value
0        1.412816e+07  4.184262e+06     1
1        1.412804e+07  4.183894e+06     1
2        1.412799e+07  4.183657e+06     1
3        1.412843e+07  4.184020e+06     1
4        1.412808e+07  4.184177e+06     1
5        1.412838e+07  4.184090e+06     1
6        1.412825e+07  4.183745e+06     1
7        1.412844e+07  4.184062e+06     1
8        1.412839e+07  4.183815e+06     1
9        1.412815e+07  4.183934e+06     1
10       1.412808e+07  4.183805e+06     1
11       1.412780e+07  4.183809e+06     1
12       1.412811e+07  4.183876e+06     1
13       1.412795e+07  4.183743e+06     1
14       1.412804e+07  4.183598e+06     1
15       1.412847e+07  4.184162e+06     1
16       1.412860e+07  4.184305e+06     1
17       1.412860e+07  4.184410e+06     1
18       1.412830e+07  4.184370e+06     1
19       1.412855e+07  4.184258e+06     1
20       1.412887e+07  4.184065e+06     1
21       1.412848e+07  4.184325e+06     1
22       1.412846e+07  4.184409e+

그러면 이제 이 geojson들로부터 읽어낸 df들을 하나로 이어주고 value 컬럼을 category로 취급하라고 알려줍니다. datashader에 필요합니다.

In [5]:
df = pd.concat(geojsons, ignore_index=True)
df.value = df.value.astype('category')

그리고 이제 hdf5로 저장합시다. 나중에 데이터 읽는 것도 빠르고 해서 좋습니다.

In [6]:
from pandas import HDFStore
hdf =HDFStore('../processed-data/korea-census-2017.h5')
hdf.put('dot_age', df, format='table', data_columns=True)

이 모든 과정이 귀찮으신 분들은 그냥 다만들어진 h5를 아래의 주소에서 다운로드받아 processed-data에 저장하시면 됩니다. 크기는 1.6GB입니다.
https://umich.box.com/s/utt6gg4goch7zy75jeipat0q7zpo3neg